In [60]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

## 1. Load

In [8]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
x_df_train = df_train.loc[:, df_train.columns != 'label']
y_df_train = df_train.loc[:, df_train.columns == 'label']
x_df_test = df_test
x_df_train.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
x_train = torch.Tensor(x_df_train.values).reshape(-1, 1, 28, 28)
y_train = torch.LongTensor(y_df_train.values).reshape(-1)
x_test = torch.Tensor(x_df_test.values).reshape(-1, 1, 28, 28)
y_test = torch.randint(0, 1, (len(x_test), ))
x_train.shape

torch.Size([42000, 1, 28, 28])

In [36]:
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## 2. Train

### 2-1. Set device

In [18]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### 2-2. Make model

In [19]:
class CustomCNN(torch.nn.Module):
    def __init__(self, inplanes=1, num_classes=10):
        super(CustomCNN, self).__init__()
        planes = 128
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(inplanes, planes, kernel_size=7, stride=1, padding=3, bias=True),
            torch.nn.BatchNorm2d(planes),
            torch.nn.ELU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            ## (batch, inplanes, 28, 28) -> (batch, planes, 14, 14)
            torch.nn.Conv2d(planes, planes, kernel_size=7, stride=1, padding=3, bias=True),
            torch.nn.BatchNorm2d(planes),
            torch.nn.ELU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
            ## (batch, planes, 14, 14) -> (batch, planes, 7, 7)
            torch.nn.Conv2d(planes, planes, kernel_size=7, stride=2, padding=3, bias=True),
            ## (batch, planes, 7, 7) -> (batch, planes, 4, 4)
            torch.nn.BatchNorm2d(planes),
            torch.nn.ELU(inplace=True),
            torch.nn.Conv2d(planes, planes, kernel_size=7, stride=2, padding=3, bias=True),
            ## (batch, planes, 4, 4) -> (batch, planes, 2, 2)
            torch.nn.BatchNorm2d(planes),
            torch.nn.ELU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
            ## (batch, planes, 2, 2) -> (batch, planes, 1, 1)
        )
        self.flatten = torch.nn.Flatten()
        ## (batch, planes, 1, 1) -> (batch, planes)
        self.classification = torch.nn.Linear(planes, num_classes, bias=True)
        ## (batch, planes) -> (batch, num_classes)


    def forward(self, x):
        out = self.conv(x)
        out = self.flatten(out)
        out = self.classification(out)
        return out

### 2-3. Set model, cost function, optimizer

In [77]:
learning_rate = 0.01
epochs = 10
model = CustomCNN(inplanes=1, num_classes=10).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### 2-4. Train model

In [78]:
model.train()
for epoch in range(1, epochs+1):
    train_loss = 0
    train_accuracy = 0
    for data, label in tqdm(train_loader):
        data, label = data.to(device), label.to(device)
        
        output = model(data)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predict = output.max(1)[1]
        train_accuracy += predict.eq(label).sum().item()
    else:
        train_loss /= len(train_loader)
        train_accuracy *= (100 / len(train_loader.dataset))
        print("Train Result Epoch = {}, Loss = {:.4f}, Accuracy = {:.4f}%)".format(epoch, train_loss, train_accuracy))

100%|██████████| 1313/1313 [00:17<00:00, 75.61it/s]
Train Result Epoch = 1, Loss = 0.1211, Accuracy = 96.7119%)
100%|██████████| 1313/1313 [00:17<00:00, 74.95it/s]
Train Result Epoch = 2, Loss = 0.0399, Accuracy = 98.9095%)
100%|██████████| 1313/1313 [00:17<00:00, 74.74it/s]
Train Result Epoch = 3, Loss = 0.0243, Accuracy = 99.4000%)
100%|██████████| 1313/1313 [00:17<00:00, 75.77it/s]
Train Result Epoch = 4, Loss = 0.0155, Accuracy = 99.6381%)
100%|██████████| 1313/1313 [00:17<00:00, 75.72it/s]
Train Result Epoch = 5, Loss = 0.0097, Accuracy = 99.8143%)
100%|██████████| 1313/1313 [00:17<00:00, 75.50it/s]
Train Result Epoch = 6, Loss = 0.0059, Accuracy = 99.9333%)
100%|██████████| 1313/1313 [00:17<00:00, 75.66it/s]
Train Result Epoch = 7, Loss = 0.0037, Accuracy = 99.9667%)
100%|██████████| 1313/1313 [00:17<00:00, 75.02it/s]
Train Result Epoch = 8, Loss = 0.0026, Accuracy = 99.9833%)
100%|██████████| 1313/1313 [00:17<00:00, 75.22it/s]
Train Result Epoch = 9, Loss = 0.0019, Accuracy = 99

### 2-5. Predict test data

In [79]:
predict = []
model.eval()
with torch.no_grad():
    test_loss = 0
    test_accuracy = 0
    for data, _ in tqdm(test_loader):
        data = data.to(device)

        output = model(data)
        predict.extend(output.max(1)[1].tolist())
len(predict)

100%|██████████| 875/875 [00:04<00:00, 196.23it/s]


28000

### 2-6. make submission.csv

In [80]:
submission = pd.DataFrame({'ImageId': [i for i in range(1, len(predict) + 1)], 'Label': predict})
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3


In [81]:
submission.to_csv("submission.csv", index=False)